Данное домашнее задание посвящено схеме базе данных, связанной с футбольным командами. Схема расположена в hw_diagram.jpg
## Описание таблиц 

### Positions 
Позиции игроков
id - уникальный идентификатор позиции 
Name - Название таблицы 

### Coaches 
Таблица тренеров 
id - уникальный идентификатор тренера
Name - имя тренера 

## Teams 
Таблица команд 
id - уникальный идентификатор  команды 
Name - имя команды 
Location - расположение команды 
Coach_id - идентификатор тренера 

### Players 
Таблица игроков 
Id - уникальный идентификатор игрока 
Name - имя игрока 
Age - возраст игрока 
Position_id - идентификатор позиции 
Team_id - идентификатор команды 
Start_date - дата начала игры в команде 

### PlayersArchive 
Архив по игрокам (информация о том, в какой команде игрок играл ранее) 
player_id - идентификатор игрока 
team_id - идентификатор команды 
start_date - дата начала игры 
end_date - дата окончания игры

In [57]:
%load_ext sql
%sql sqlite:///chinook.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: None@chinook.db'

In [2]:
import sqlite3

In [3]:
db = sqlite3.connect('chinook.db')

# Задание 1
Реализовать перенакатываемые sql-скрипты по схеме hw_diagram, приложенного ниже.

In [70]:
%%sql
DROP TABLE if exists Players;
DROP TABLE if exists Teams;
DROP TABLE if exists Coaches;
DROP TABLE if exists Positions;
DROP TABLE if exists PlayersArchive;

CREATE TABLE PlayersArchive (
    Player_id integer,
    Team_id integer,
    start_date date,
    end_date date,
    primary key(Player_id, Team_id)
);

CREATE TABLE Positions (
    id integer primary key,
    Name varchar(50)
);

CREATE TABLE Coaches (
     id integer primary key,
     Name varchar(50)
);

CREATE TABLE Teams (
    id integer primary key,
    Name varchar(50),
    Location integer,
    Coach_id integer,
    foreign key (Coach_id) references Coaches(id)
);

CREATE TABLE Players (
    id integer primary key,
    Name varchar(50),
    Age integer,
    Position_id integer,
    Team_id integer,
    Start_date date,
    foreign key (Position_id) references Positions(id),
    foreign key (Team_id) references Teams(id),
    foreign key (id) references PlayersArchive(Player_id)
);


Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [5]:
%%sql
PRAGMA foreign_keys = ON;

Done.


[]

# Задание 2
Написать sql-скрипт, осуществляюший добавление всех возможных позиций игроков. (В данной задаче можно ограничится четырьмя - вратарь, защитник, полузащитник, нападающий)

In [71]:
%%sql
INSERT INTO Positions VALUES
(1, 'Goalkeeper'),
(2, 'Defender'),
(3, 'Midfielder'),
(4, 'Forward');

4 rows affected.


[]

# Задание 3
Написать функцию, осуществляющую добавление тренера. На вход функции подается имя тренера. При невозможности вставки функция должна обработать данное исключение и вывести пользователю информацию о проблеме

После написания функции необходимо добавить несколько тренеров для таблицы.

In [72]:
class MyException(Exception):
    '''Мой класс ошибок'''
    pass
   
def new_coach(new_coaches):
    cursor = db.cursor()
    try:
        cursor.execute( '''SELECT COALESCE(MAX(id)+1, 1) AS new_id FROM Coaches''')
        new_id = cursor.fetchone()[0]
        for i, coach in enumerate(new_coaches):
            if coach.isalpha() == 0:
                raise MyException(Exception)
            cursor.execute('''
            INSERT INTO Coaches(id, name) VALUES(?,?)''', (new_id + i, coach))
        
        db.commit()        
    except MyException as ex:
        print('The name contains not only letters')
        return -2
    except TypeError:
        print('Error')
        return -1

In [73]:
new_coach(['Slutskiy', 'Gancharenko'])
new_coach(['Manchini'])
new_coach(['Karera'])
new_coach(['Cherchesov100', 'Berdiev,,', 'Kaprin'])

The name contains not only letters


-2

In [74]:
%%sql 
SELECT *
FROM Coaches

Done.


id,Name
1,Slutskiy
2,Gancharenko
3,Manchini
4,Karera


# Задание 4

Написать функцию, осуществляющую добавление команды. На вход функция принимает (имя команды, местоположение, имя тренера).
На выходе должна быть добавлена команда.

После написания функции необходимо добавить несколько команд

In [75]:
def find_coach(Coach):
    cur = db.cursor()
    cur.execute('''SELECT id FROM Coaches WHERE Name = ?''', (Coach,))
    try:
        coach_id = cur.fetchone()[0]
        return coach_id
    except TypeError:
        print('Error')
        return -1
    
def insert_team(Name, Location, Coach):
    cur = db.cursor();
    cur.execute('''SELECT COALESCE(MAX(id)+1, 1) FROM Teams''')
    id = cur.fetchone()[0]  
    Coach_id = find_coach(Coach)
    cur.execute('''INSERT INTO Teams(id, Name, Location, Coach_id) values(?,?,?,?)''', (id, Name, Location, Coach_id))
    db.commit()  

In [76]:
insert_team('CSKA', 'Moscow', 'Gancharenko')
insert_team('Zenith', 'St.Petersburg', 'Manchini')
insert_team('Spartak','Moscow','Karera')
insert_team('Krasnodar', 'Krasnodar', 'Shalimov')


Error


In [77]:
%%sql
SELECT *
FROM Teams

Done.


id,Name,Location,Coach_id
1,CSKA,Moscow,2
2,Zenith,St.Petersburg,3
3,Spartak,Moscow,4
4,Krasnodar,Krasnodar,-1


# Задание 5

Написать функцию, осуществляющую обновление информации о команде. У функции на входе должен быть один обязательный параметр - название команды и два необязательных параметра - имя тренера и местоположение. При этом хотя бы один из необязательных параметров должен быть заполнен. Функция должна обновлять значение переданных необязательных параметров.

После написания функции вызовите данную функцию несколько раз

In [78]:
def update_team(Name, Coach = None, Location = None):
    cur = db.cursor()
    if Coach == None and Location == None:
        print('Error: There must be at least one of the optional parameters')
        return -1
    if Coach == None and Location:
        cur.execute('''UPDATE Teams SET Location = ? WHERE Name = ?''',(Location, Name))
    if Coach and Location == None:
        coach_id = find_coach(Coach)
        cur.execute('''UPDATE Teams SET Coach_id = ? WHERE Name = ?''',(coach_id, Name))
    if Coach and Location:
        coach_id = find_coach(Coach)
        cur.execute('''UPDATE Teams SET Location = ? WHERE Name = ?''',(Location, Name))
        cur.execute('''UPDATE Teams SET Coach_id = ? WHERE Name = ?''',(coach_id, Name))
    db.commit()        
        

In [79]:
update_team('Krasnodar')

Error: There must be at least one of the optional parameters


-1

In [80]:
update_team('CSKA', Coach = 'Slutskiy')

In [81]:
update_team('Spartak', Location = 'Ufa')

In [82]:
update_team('Zenith', Coach = 'Gancharenko', Location = 'Vladivostok')


In [83]:
%sql SELECT * FROM Teams

Done.


id,Name,Location,Coach_id
1,CSKA,Moscow,1
2,Zenith,Vladivostok,2
3,Spartak,Ufa,4
4,Krasnodar,Krasnodar,-1


# Задание 6

Написать функцию, осуществляющую добавление нового футболиста в таблицу игроков. На входе (имя, возраст, название позиции, название команды, дата перехода в команду(необязательный параметр)). Если дата перехода не указана, то нужно использовать текущую дату. Предусмотреть различные ошибки (такие как неверное имя позиции, неверное название команды, возраст < 10 и > 50 и т.п.)

После написания функции необходимо добавить несколько игроков.

In [86]:
def new_player(Name, Age, Position, Team, Date = None):
    cur = db.cursor()
    cur.execute('''SELECT COALESCE(MAX(id) + 1, 1) FROM Players''')
    try:
        id = cur.fetchone()[0]
        cur.execute('''SELECT id FROM Positions WHERE Name = ?''',(Position,))
        position_id = cur.fetchone()[0]
        cur.execute('''SELECT id FROM Teams WHERE Name = ?''',(Team,))
        team_id = cur.fetchone()[0]
        cur.execute('''SELECT id FROM Positions ''');
        pos = cur.fetchall()
        cur.execute('''SELECT id FROM Teams ''');
        tms = cur.fetchall()
        if Date == None:
            cur.execute('''SELECT date(current_date)''')
            Date = cur.fetchone()[0]
          
        if ((position_id,) in pos and (Age < 50 and Age > 10) and (team_id,) in tms):
            cur.execute('''INSERT INTO Players (id, Name, Age, Position_id, Team_id, Start_date) values(?,?,?,?,?,?)''',
                        (id, Name, Age, position_id, team_id, Date))
        else:
            print('Error')
        db.commit()
    except TypeError: 
        print('Error-')

In [87]:
new_player('Golovin', 21, 'Midfielder', 'CSKA')
new_player('Glushakov', 31, 'Midfielder', 'Spartak', '2012-04-16')

In [88]:
%sql SELECT * from Players

Done.


id,Name,Age,Position_id,Team_id,Start_date
1,Golovin,21,3,1,2018-04-13
2,Glushakov,31,3,3,2012-04-16


# Задание 7
Написать функцию для перевода игрока в другую команду. Данная функция должна осуществить архивирование текущей записи в таблице игроков в таблицу players_archive с заполнение end_date - датой начала игры за новую команду - 1 день. А также осуществлять изменение поля в таблице Players. При возникновении ошибки должен осуществляться полный откат транзакции.

In [89]:
def update_player(Name, new_team, Date): 
    cur = db.cursor() 
    cur.execute('''SELECT COALESCE(MAX(Player_id)+1, 1) FROM PlayersArchive''')
    pl_id = cur.fetchone()[0]
    cur.execute( '''SELECT Date(?, '-1 day')''',(Date,)) 
    d = cur.fetchone()[0] 
    try: 
        cur.execute( '''SELECT Start_date FROM Players WHERE Name = ?''',(Name,))
        old_d = cur.fetchone()[0]
        cur.execute( '''SELECT team_id FROM Players WHERE Name = ?''',(Name,)) 
        t = cur.fetchone()[0] 
        cur.execute('''INSERT INTO PlayersArchive(Player_id, Team_id, start_date, end_date) VALUES(?,?,?,?)''', (pl_id, t, old_d, d)) 
        cur.execute( '''SELECT id FROM teams WHERE Name = ?''',(new_team,)) 
        t_id = cur.fetchone()[0] 
        cur.execute( '''SELECT id FROM Players WHERE Name = ?''',(Name,)) 
        id = cur.fetchone()[0] 
        cur.execute ('''UPDATE Players set team_id = ?, start_date= ? WHERE Name = ?''', (t_id, Date, Name)) 
        db.commit() 
    except TypeError: 
        print('ERROR') 
        db.rollback()

In [90]:
%sql SELECT * FROM Players

Done.


id,Name,Age,Position_id,Team_id,Start_date
1,Golovin,21,3,1,2018-04-13
2,Glushakov,31,3,3,2012-04-16


In [91]:
update_player('Golovin', 'Zenith', '2018-05-12')

In [92]:
%sql SELECT * FROM PlayersArchive

Done.


Player_id,Team_id,start_date,end_date
1,1,2018-04-13,2018-05-11
